### Loading LLM and Dataset

In [1]:
import json
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

musique_dataset = []
with open("./data/musique_ans_v1.0_dev.jsonl","r") as f:
    for k in f.readlines():
        musique_dataset.append(json.loads(k))

In [1]:
from datasets import load_from_disk
a = load_from_disk("/home/tkdrnjs0621/work/multihopqa_v1/inference/subset_300/question_decomposition")
print(a[0])

{'id': '2hop__460946_294723', 'paragraphs': [{'idx': 0, 'title': "Grant's First Stand", 'paragraph_text': 'Grant\'s First Stand is the debut album by American jazz guitarist Grant Green featuring performances by Green recorded and released on the Blue Note label in 1961. Earlier recordings made by Green for Blue Note were released as "First Session" in 2001.', 'is_supporting': False}, {'idx': 1, 'title': 'List of show business families', 'paragraph_text': 'Actress / director / singer Phylicia Rashād is the older sister of performer Debbie Allen, who is married to former NBA basketball player, Norm Nixon. Phylicia Rashād is the former spouse of both Victor Willis, former lead singer of the group Village People, and former NFL football player turned sportscaster, Ahmad Rashād. Phylicia and Ahmad Rashād are the parents of actress Condola Rashād.', 'is_supporting': False}, {'idx': 2, 'title': 'Green', 'paragraph_text': 'For the ancient Egyptians, green had very positive associations. The h

In [2]:
print(a[0]["question"])
print(a[0]["output"])

Who is the spouse of the Green performer?
<Q> Who is the Green performer?
<A> $1

<Q> Who is the spouse of $1?
<A> $ANSWER


In [2]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [20]:
from datasets import Dataset

d = Dataset.from_dict({"foo": [(1, 2, 3), (5, 7, 8)]})
def func(x):
    # print(x)
    return {"foo": [i for row in x["foo"] for i in row]}
    # return {"foo": [i for row in x["foo"] for i in row]}
d = d.map(func, batched=True)

print(d["foo"])

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

NameError: name 'row' is not defined

In [26]:
from datasets import Dataset

A = Dataset.from_list(musique_dataset)
# Assuming A is loaded as a Hugging Face dataset
def flatten_dataset(A):
    # Lists to store flattened data
    original_ids = []
    titles = []
    paragraph_texts = []

    # Iterate through each row in dataset A
    for record in A:
        original_id = record["id"]
        
        # Flatten the paragraphs
        for paragraph in record["paragraphs"]:
            original_ids.append(original_id)
            titles.append(paragraph["title"])
            paragraph_texts.append(paragraph["paragraph_text"])
    
    # Create the flattened dataset B
    B = Dataset.from_dict({
        "original_id": original_ids,
        "title": titles,
        "paragraph_text": paragraph_texts
    })
    
    return B

# Example usage
# A = Dataset.load_from_disk("path_to_A")
B = flatten_dataset(A)

In [7]:
from datasets import load_from_disk
a = load_from_disk("/home/tkdrnjs0621/work/multihopqa_v1/inference/subset_2/context_conversion")
print(len(a))
print(a[0]['paragraph_text'])

40
Grant's First Stand is the debut album by American jazz guitarist Grant Green featuring performances by Green recorded and released on the Blue Note label in 1961. Earlier recordings made by Green for Blue Note were released as "First Session" in 2001.


In [27]:
print(B[0])

{'original_id': '2hop__460946_294723', 'title': "Grant's First Stand", 'paragraph_text': 'Grant\'s First Stand is the debut album by American jazz guitarist Grant Green featuring performances by Green recorded and released on the Blue Note label in 1961. Earlier recordings made by Green for Blue Note were released as "First Session" in 2001.'}


### Predefined ICL Examples

In [4]:
messages_passage_summarization = [{"role": "system", "content": """You are a language model that paraphrases the given text in an entity-based manner. Named entities should be extracted, and then the information about the entities should be listed. Note that the informations should cover all given text about the entity, so many information should be repetitive."""},
    {"role": "user", "content": """[Topic]
Darlene Remembers Duke, Jonathan Plays Fats
[Text]
Darlene Remembers Duke, Jonathan Plays Fats is a 1982 album by Jo Stafford and Paul Weston in which they perform in character as Jonathan and Darlene Edwards. The duo put their own unique interpretation on the music of Duke Ellington and Fats Waller with Stafford singing deliberately off key, while Weston plays an out of tune piano. The album was issued by Corinthian Records (COR-117). "Billboard" reviewed the album when it was newly released, saying, "the sounds they achieve may well lead to another Grammy for the duo next year." Stafford and Weston, in their personas of Jonathan and Darlene Edwards, were interviewed by "Los Angeles Magazine" following the release of the album."""},
    {"role": "assistant", "content": """<Darlene Remembers Duke, Jonathan Plays Fats>: Darlene Remembers Duke, Jonathan Plays Fats is a 1982 album. Darlene Remembers Duke, Jonathan Plays Fats is by Jo Stafford and Paul Weston. Darlene Remembers Duke, Jonathan Plays Fats features performances in character as Jonathan and Darlene Edwards. Darlene Remembers Duke, Jonathan Plays Fats includes unique interpretations of Duke Ellington's and Fats Waller's music. Darlene Remembers Duke, Jonathan Plays Fats has Jo Stafford singing deliberately off-key. Darlene Remembers Duke, Jonathan Plays Fats has Paul Weston playing an out-of-tune piano. Darlene Remembers Duke, Jonathan Plays Fats was issued by Corinthian Records with the catalog number COR-117. Darlene Remembers Duke, Jonathan Plays Fats was reviewed by "Billboard" on its release. Darlene Remembers Duke, Jonathan Plays Fats was discussed by Stafford and Weston in an interview with "Los Angeles Magazine."

<Jo Stafford>: Jo Stafford performed on Darlene Remembers Duke, Jonathan Plays Fats. Jo Stafford performed as the character Darlene Edwards on Darlene Remembers Duke, Jonathan Plays Fats. Jo Stafford sang deliberately off-key on Darlene Remembers Duke, Jonathan Plays Fats. Jo Stafford was interviewed by "Los Angeles Magazine" about Darlene Remembers Duke, Jonathan Plays Fats.

<Paul Weston>: Paul Weston performed on Darlene Remembers Duke, Jonathan Plays Fats. Paul Weston performed as the character Jonathan Edwards on Darlene Remembers Duke, Jonathan Plays Fats. Paul Weston played an out-of-tune piano on Darlene Remembers Duke, Jonathan Plays Fats. Paul Weston was interviewed by "Los Angeles Magazine" about Darlene Remembers Duke, Jonathan Plays Fats.

<Duke Ellington>: Duke Ellington's music was interpreted on Darlene Remembers Duke, Jonathan Plays Fats.

<Fats Waller>: Fats Waller's music was interpreted on Darlene Remembers Duke, Jonathan Plays Fats.

<Corinthian Records>: Corinthian Records issued Darlene Remembers Duke, Jonathan Plays Fats. Corinthian Records issued Darlene Remembers Duke, Jonathan Plays Fats with catalog number COR-117.

<Billboard>: "Billboard" reviewed Darlene Remembers Duke, Jonathan Plays Fats on its release. "Billboard" commented that the album's unique sounds might earn the duo another Grammy.

<Los Angeles Magazine>: "Los Angeles Magazine" interviewed Jo Stafford and Paul Weston following the release of Darlene Remembers Duke, Jonathan Plays Fats. "Los Angeles Magazine" featured an interview with Stafford and Weston in their personas as Jonathan and Darlene Edwards."""},
    {"role": "user", "content": """[Topic]
Felix R. de Zoysa
[Text]
Felix R. de Zoysa is the founding Chairman of Stafford Motor Company, Stafford International School and Atlas Hall in Sri Lanka. De Zoysa was the head of Auto & General Agencies who held the distributorship of Honda motor vehicles in Sri Lanka in 1960s. After the incorporation of both companies, Stafford Motor Company became the sole distributor for Honda motor vehicles. He was also nicknamed "Mr. Honda" of Sri Lanka."""},
    {"role": "assistant", "content": """<Felix R. de Zoysa>: Felix R. de Zoysa is the founding Chairman of Stafford Motor Company. Felix R. de Zoysa is the founding Chairman of Stafford International School. Felix R. de Zoysa is the founding Chairman of Atlas Hall in Sri Lanka. Felix R. de Zoysa was the head of Auto & General Agencies. Felix R. de Zoysa held the distributorship of Honda motor vehicles in Sri Lanka in the 1960s. Felix R. de Zoysa, after the incorporation of both companies, saw Stafford Motor Company become the sole distributor for Honda motor vehicles. Felix R. de Zoysa was nicknamed "Mr. Honda" of Sri Lanka.

<Stafford Motor Company>: Stafford Motor Company was founded by Felix R. de Zoysa. Stafford Motor Company became the sole distributor for Honda motor vehicles after the incorporation of both companies.

<Stafford International School>: Stafford International School was founded by Felix R. de Zoysa.

<Atlas Hall>: Atlas Hall was founded by Felix R. de Zoysa in Sri Lanka.

<Auto & General Agencies>: Auto & General Agencies was headed by Felix R. de Zoysa. Auto & General Agencies held the distributorship of Honda motor vehicles in Sri Lanka in the 1960s.

<Honda motor vehicles>: Honda motor vehicles had their distributorship held by Auto & General Agencies in Sri Lanka in the 1960s. Honda motor vehicles were solely distributed by Stafford Motor Company after the incorporation of both companies.

<Sri Lanka>: Sri Lanka is the country where Felix R. de Zoysa was the founding Chairman of Stafford Motor Company. Sri Lanka is where Felix R. de Zoysa was the founding Chairman of Stafford International School. Sri Lanka is where Felix R. de Zoysa was the founding Chairman of Atlas Hall. Sri Lanka is where Felix R. de Zoysa held the distributorship of Honda motor vehicles through Auto & General Agencies in the 1960s. Sri Lanka is where Stafford Motor Company became the sole distributor for Honda motor vehicles after the incorporation of both companies. Sri Lanka is where Felix R. de Zoysa was nicknamed "Mr. Honda."

<Mr. Honda>: Mr. Honda of Sri Lanka is a nickname of Felix R. de Zoysa, who is the founding Chairman of Stafford Motor Company, Stafford International School and Atlas Hall in Sri Lanka."""},
    {"role": "user", "content": """[Topic]
Stafford Mills
[Text]
Stafford Mills is an historic textile mill complex located on County Street in Fall River, Massachusetts, USA. Founded in 1872, it is a well-preserved late-19th century textile complex, typical of the mills built in Fall River during its period of most rapid growth. It is noted in particular for its exceptionally fine Romanesque brick office building. The complex was added to the National Register of Historic Places in 1983."""},
    {"role": "assistant", "content": """<Stafford Mills>: Stafford Mills is a historic textile mill complex. Stafford Mills is located on County Street in Fall River, Massachusetts, USA. Stafford Mills was founded in 1872. Stafford Mills is a well-preserved late-19th century textile complex. Stafford Mills is typical of the mills built in Fall River during its period of most rapid growth. Stafford Mills is known for its exceptionally fine Romanesque brick office building. Stafford Mills was added to the National Register of Historic Places in 1983.

<County Street>: County Street is the location of Stafford Mills in Fall River, Massachusetts, USA.

<Fall River>: Fall River is where Stafford Mills is located in Massachusetts, USA. Fall River is where Stafford Mills, a late-19th century textile complex, was built during a period of rapid growth. Fall River is where the National Register of Historic Places includes Stafford Mills.

<Massachusetts>: Massachusetts is the state in the USA where Stafford Mills is located in Fall River. Massachusetts includes Stafford Mills, a historic textile mill complex added to the National Register of Historic Places.

<USA>: The USA is the country where Stafford Mills is located in Fall River, Massachusetts.

<Romanesque Brick Office Building>: The Romanesque brick office building is a notable feature of Stafford Mills. The Romanesque brick office building contributes to Stafford Mills’ recognition as a well-preserved late-19th century textile complex.

<National Register of Historic Places>: The National Register of Historic Places included Stafford Mills in 1983."""},
]

In [5]:
messages_query_decomposition = [{"role": "system", "content": """You are a language model that divides the given query into list of subqueries and answers. Answers must be in form of $1,$2,$3, ... and the final answer must be $ANSWER."""},
    {"role": "user", "content": """"Where is the school that employs Charles Stafford located?"""},
    {"role": "assistant", "content": """<Q> Which school employs Charles Stafford?
<A> $1

<Q> Where is $1 located?
<A> $ANSWER"""},
    {"role": "user", "content":"""What year saw the creation of the region where the county of Hertfordshire is located?"""},
    {"role": "assistant", "content": """<Q> Where is Hertfordshire located?
<A> $1

<Q> In what region is $1 located? 
<A> $2

<Q> In what year was $2 created?
<A> $ANSWER"""},
    {"role": "user", "content":"""Are attendance remittances that important to the league whose trophy was won by an undefeated Arsenal team in the 2003-2004 season?"""},
    {"role": "assistant", "content": """
<Q> Which league's trophy was won by an undefeated Arsenal team in the 2003-2004 season?
<A> $1

<Q> Are attendance remittances important to $1?
<A> $ANSWER"""}, 
    {"role": "user", "content":"""What is something that is not consistent between the Jewish population in the country know for its Muslims and the people who traded slaves into the Middle East?"""},
    {"role": "assistant", "content": """<Q> What country is known for its Muslim population? <A> $1

<Q> What is the Jewish population in $1? <A> $2

<Q> Who were the people who traded slaves into the Middle East? <A> $3

<Q> What is something that is not consistent between the Jewish population in $1 and $3? <A> $ANSWER"""}, 
    {"role": "user", "content":"""Maria Leopoldina of the country where the Botanical Garden of where Alfred Biasiadeck was educated is the child of?"""},
    {"role": "assistant", "content": """<Q> Where was Alfred Biasiadeck educated? <A> $1

<Q> Where is the Botanical Garden of $1 located? <A> $2

<Q> Which country is $2 located in? <A> $3

<Q> Who is Maria Leopoldina of $3 the child of? <A> $ANSWER"""}, 
]

In [36]:
messages_question_answering = [{"role": "system", "content":"""You are a language model that answers the given question using the given knowledge. The answer must be a single term that can be the answer of the question, without auxiliary terms."""},
{"role":"user","content":"""[Knowledge]
Hertfordshire is the county immediately north of London and is part of the East of England region, a mainly statistical unit. A significant minority of the population across all districts are City of London commuters. To the east is Essex, to the west is Buckinghamshire and to the north are Bedfordshire and Cambridgeshire.
[Question]
Where is the county of Hertfordshire located?
"""},
{"role":"assistant","content":"""England"""},
# {"role":"user","content":"""[Knowledge]
# The journal was established in December 1984 by founding editor-in-chief William J. Whelan under the auspices of the International Union of Biochemistry and Molecular Biology. Adam S. Wilkins became editor in January 1990. Originally published by ICSU Press and The Company of Biologists, "BioEssays" has been published by John Wiley & Sons since January 1998. Andrew Moore became editor-in-chief in August 2008.
# [Question]
# Who is the employer of John J. Collins"""},
# {"role":"assistant","content":"""None"""},
{"role":"user","content":"""[Knowledge]
The Game is a 1997 American mystery thriller film directed by David Fincher, starring Michael Douglas and Sean Penn, and produced by Propaganda Films and PolyGram Filmed Entertainment. It tells the story of a wealthy investment banker who is given a mysterious gift: participation in a game that integrates in strange ways with his everyday life. As the lines between the banker's real life and the game become more uncertain, hints of a large conspiracy become apparent.
[Question]
What studio distributed The Game?
"""},
{"role":"assistant","content":"""PolyGram Filmed Entertainment"""},]

In [7]:
import re
data = musique_dataset[1]

knowledge_list = {}
for a in data["paragraphs"]:
    title = a["title"]
    paragraph = a["paragraph_text"]
    txt = f"""[Topic]\n{title}\n[Text]\n{paragraph}"""
    messages = messages_passage_summarization+[{"role":'user',"content":txt}]

    outputs = pipeline(
        messages,
        max_new_tokens=256,
    )
    out = outputs[0]["generated_text"][-1]["content"]
    pattern = r"<([^>]+)>: ([^\n]+)"

    result = re.findall(pattern, out)
    for a in result:
        knowledge_list[a[0]]=a[1]

for k,v in knowledge_list.items():
    print(k)
    print(v)
    print()


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_to

SICRAL 1B
SICRAL 1B is a military communications satellite. SICRAL 1B was built by Thales Alenia Space. SICRAL 1B is for the Italian Armed Forces. SICRAL 1B is a dual-use spacecraft. SICRAL 1B is based on the Italsat 3000 bus. SICRAL 1B includes one EHF/Ka band, three UHF-band, and five active SHF-band transponders. SICRAL 1B is designed to be operable for 13 years.

Thales Alenia Space
Thales Alenia Space built SICRAL 1B, a military communications satellite for the Italian Armed Forces.

Italian Armed Forces
The Italian Armed Forces are the users of SICRAL 1B, a military communications satellite built by Thales Alenia Space.

Italian defense ministry
The Italian defense ministry will use some of the satellite's transmission capacity of SICRAL 1B.

NATO
NATO will use some of the satellite's transmission capacity of SICRAL 1B.

Telespazio
Telespazio will use some of the satellite's transmission capacity of SICRAL 

Salix arbuscula
Salix arbuscula is a plant species. Salix arbuscula is a

In [9]:
txt = data["question"]
messages = messages_query_decomposition+[{"role":'user',"content":txt}]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
out = outputs[0]["generated_text"][-1]["content"]
pattern = r"<Q>\s*(.*?)\s*<A>\s*(.*?)\s*(?=<Q>|$)"

# Find all matches of the pattern in the data
qa_pairs = re.findall(pattern, out)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [10]:
print(qa_pairs)

[('Which company distributed the film UHF?', '$1'), ('Who founded $1?', '$ANSWER')]


In [18]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load the Contriever model and tokenizer
model_name = "facebook/contriever"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Encode text function
def encode_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        return model(**inputs).last_hidden_state.mean(dim=1)

def retrieve_knowledge(knowledge_dictionary, query_question):
    texts = list(knowledge_dictionary.keys())

    # Encode query and texts
    query_embedding = encode_text(query_question)
    text_embeddings = torch.vstack([encode_text(text) for text in texts])

    # Find the most similar text
    similarities = torch.nn.functional.cosine_similarity(query_embedding, text_embeddings)
    top_text = texts[similarities.argmax().item()]
    top_text2 = list(knowledge_dictionary.values())[similarities.argmax().item()]
    return top_text2

/home/tkdrnjs0621/miniconda3/envs/torch_241/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [39]:
defined_list = {}
for qa in qa_pairs:
    question = qa[0]
    for k,v in defined_list.items():
        question = question.replace(k,v)
    knowledge = retrieve_knowledge(knowledge_list, question)
    txt = f"""[Knowledge]\n{knowledge}\n[Question]\n{question}"""
    messages = messages_question_answering+[{"role":'user',"content":txt}]
    outputs = pipeline(
        messages,
        max_new_tokens=256,
    )
    out = outputs[0]["generated_text"][-1]["content"]
    defined_list[qa[1]]=out
    print("tmp:",question,out)
    print("knowledge",knowledge)
print(defined_list)
print(defined_list["$ANSWER"])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


tmp: Which company distributed the film UHF? Orion Pictures
knowledge UHF is a film written by Yankovic and Levey. UHF is based on the music of "Weird Al" Yankovic. UHF was made after Yankovic's second studio album. UHF features parodies and comedy. UHF has a straight man character named George, who has a vivid imagination. UHF's straight man character is used to support the inclusion of parodies within the film. UHF was made to apply Yankovic's parody and comedy to film. UHF was produced with the approach of using George's vivid imagination to create parodies. UHF was added to include parodies within the film. UHF was filmed in a way that supports the inclusion of parodies. UHF was made with the intention of including parodies.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


tmp: Who founded Orion Pictures? Mike Medavoy
knowledge Orion Pictures was co-founded by Mike Medavoy in 1978.
{'$1': 'Orion Pictures', '$ANSWER': 'Mike Medavoy'}
Mike Medavoy


In [9]:
from datasets import load_from_disk, concatenate_datasets


a = load_from_disk("/home/tkdrnjs0621/work/multihopqa_v1/inference/subset_600/context_conversion")
b = load_from_disk("/home/tkdrnjs0621/work/multihopqa_v1/inference/subset_601_1200/context_conversion")
c = load_from_disk("/home/tkdrnjs0621/work/multihopqa_v1/inference/subset_1201_end/context_conversion")

print(len(a),len(b),len(c))
combined_dataset = concatenate_datasets([a,b,c])
print(len(combined_dataset))

11998 12000 24297
48295


In [10]:
combined_dataset.save_to_disk("/home/tkdrnjs0621/work/multihopqa_v1/inference/musique_ans/context_conversion")

Saving the dataset (0/3 shards):   0%|          | 0/48295 [00:00<?, ? examples/s]

In [11]:

a = load_from_disk("/home/tkdrnjs0621/work/multihopqa_v1/inference/musique_ans/context_conversion")

In [20]:
print(len(a["original_id"]),len(set(a["original_id"])))
print(set(a["original_id"]) - set(b["id"]))
print(set(b["id"])-set(a["original_id"]))

48295 2416
set()
{'3hop1__488744_443779_52195'}
1200


In [15]:

b = load_from_disk("/home/tkdrnjs0621/work/multihopqa_v1/inference/musique_ans/question_decomposition_raw")
print(len(b))

2417


In [29]:
x600 = load_from_disk("/home/tkdrnjs0621/work/multihopqa_v1/inference/subset_600s/context_conversion")
x1200 = load_from_disk("/home/tkdrnjs0621/work/multihopqa_v1/inference/subset_1200s/context_conversion")
x0_600 = load_from_disk("/home/tkdrnjs0621/work/multihopqa_v1/inference/tmp/subset_600/context_conversion")
x601_1200 = load_from_disk("/home/tkdrnjs0621/work/multihopqa_v1/inference/tmp/subset_601_1200/context_conversion")
x1201_end = load_from_disk("/home/tkdrnjs0621/work/multihopqa_v1/inference/tmp/subset_1201_end/context_conversion")
adecomp = load_from_disk("/home/tkdrnjs0621/work/multihopqa_v1/inference/musique_ans/question_decomposition_raw")

In [40]:
print(x0_600[-1])
print(x600[0])
print(x601_1200[0])
print("1200 first",x1200[0])
print("1200 last",x1200[-1])
print(x1201_end[0])
print(x1201_end[-1])
print(adecomp[600]["id"],adecomp[1200]["id"],adecomp[1201]["id"],adecomp[-1]["id"])

{'original_id': '2hop__88632_67668', 'title': 'Barney Stinson', 'paragraph_text': "Barney Stinson is one of five main characters on How I Met Your Mother. He is a manipulative, oversexed businessman in his thirties who always wears a suit, likes women with ``daddy issues ''and is frequently willing to offer his (sometimes hypocritical) opinion. Throughout the earlier seasons, Barney is a huge womanizer, and has been described as a`` high - functioning sociopath'' by his best friend, Ted Mosby (Josh Radnor). Barney has a plethora of strategies and rules designed to meet women, sleep with them, and discard them. Through several seasons of the show, four (4) of the main characters are couples, as Ted began dating Robin Scherbatsky (Cobie Smulders) and Ted's roommate Marshall Eriksen (Jason Segel) becomes engaged and later married to Lily Aldrin (Alyson Hannigan). This leaves Barney the only single character, and, according to Harris, Barney is ``resentful ''that the other characters have 

In [41]:
combined_dataset = concatenate_datasets([x0_600,x601_1200,x1200,x1201_end])


In [42]:
print(len(combined_dataset))

48315


In [43]:
combined_dataset.save_to_disk("/home/tkdrnjs0621/work/multihopqa_v1/inference/musique_ans/context_conversion")

Saving the dataset (0/3 shards):   0%|          | 0/48315 [00:00<?, ? examples/s]

In [45]:
print(adecomp[10]["output"])

<Q> Which team plays in Stadio Ciro Vigorito?
<A> $1

<Q> In which league does $1 play?
<A> $ANSWER


In [60]:
def f(row):
    pattern = r"<([^>]+)>: ([^\n]+)"
    data = row["output"]
    qa_pairs = re.findall(pattern, data)
    row["context"]=qa_pairs
    return row
combined_dataset.map(f).select_columns(["original_id","context","title","paragraph_text","output"]).push_to_hub("tkdrnjs0621/musique_ans_processed",config_name="context_conversion",split="test")

Map:   0%|          | 0/48315 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/49 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/tkdrnjs0621/musique_ans_processed/commit/3668a77e7c8cd6da1233e54357c9ec2f54518990', commit_message='Upload dataset', commit_description='', oid='3668a77e7c8cd6da1233e54357c9ec2f54518990', pr_url=None, pr_revision=None, pr_num=None)

In [61]:
def f(row):
    pattern = r"<Q>\s*(.*?)\s*<A>\s*(.*?)\s*(?=<Q>|$)"
    data = row["output"]
    qa_pairs = re.findall(pattern, data)
    row["qa_pairs"]=qa_pairs
    return row
adecomp.map(f).select_columns(["id","qa_pairs","question","answer","output"]).push_to_hub("tkdrnjs0621/musique_ans_processed",config_name="question_decomposition",split="test")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/510 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/tkdrnjs0621/musique_ans_processed/commit/04ab23ab72a2c91a9843bf54c51c46099e39f8ac', commit_message='Upload dataset', commit_description='', oid='04ab23ab72a2c91a9843bf54c51c46099e39f8ac', pr_url=None, pr_revision=None, pr_num=None)

In [71]:
from datasets import load_dataset, Dataset
from collections import defaultdict
from tqdm import tqdm

# Load your dataset (replace "username/dataset_name" with your actual dataset path)
dataset = load_dataset("tkdrnjs0621/musique_ans_processed", name="context_conversion", split="test")

# Dictionary to accumulate contexts by original_id
merged_data = defaultdict(list)

# Group contexts by "original_id" without filtering repeatedly
for row in tqdm(dataset):
    merged_data[row["original_id"]].extend(row["context"])

# Convert the grouped data to a format suitable for Dataset
final = {
    "id": list(merged_data.keys()),
    "merged_context": list(merged_data.values())
}
final = Dataset.from_dict(final).push_to_hub("tkdrnjs0621/musique_ans_processed",config_name="merged_context_conversion",split="test")
print(final[0])
# Group by 'original_id' and merge the context lists
# grouped_dataset = Dataset.from_pandas(dataset.to_pandas().groupby("original_id", as_index=False).agg({"context": "sum"}))

# Convert back to a standard Dataset (optional)
# flattened_dataset = grouped_dataset.rename_column("original_id","id").rename_column("context","merged_context").select_columns(["id","merged_context"])

100%|██████████| 48315/48315 [00:03<00:00, 13984.44it/s]


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/981 [00:00<?, ?B/s]

h


In [73]:
from datasets import Dataset, DatasetDict

dataset1 = load_dataset("tkdrnjs0621/musique_ans_processed", name="merged_context_conversion", split="test")
dataset2 = load_dataset("tkdrnjs0621/musique_ans_processed", name="question_decomposition", split="test")
dataset2_dict = {row["id"]: row for row in dataset2}

def merge_rows(row):
    matching_row = dataset2_dict.get(row["id"], {})
    return {**row, **matching_row}

merged_dataset = dataset1.map(merge_rows)

columns_to_remove = [col for col in merged_dataset.column_names if col.endswith("_1")]
if columns_to_remove:
    merged_dataset = merged_dataset.remove_columns(columns_to_remove)

Map:   0%|          | 0/2417 [00:00<?, ? examples/s]

In [80]:
merged_dataset.select_columns(["id","merged_context",'qa_pairs','question','answer']).push_to_hub("tkdrnjs0621/musique_ans_processed2")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/tkdrnjs0621/musique_ans_processed2/commit/79cf91e6650ba5cf772411a511b2d7e99b4c552c', commit_message='Upload dataset', commit_description='', oid='79cf91e6650ba5cf772411a511b2d7e99b4c552c', pr_url=None, pr_revision=None, pr_num=None)

In [70]:
print(len(setfinal))

2417


In [ ]:
print(flattened_dataset[0])

In [55]:
import re
data = "<Q> Q1 <A> Afefe1\n\n\n<Q> Q2 fef<A> A2"

# Regular expression pattern to match each question and answer pair
pattern = r"<Q>\s*(.*?)\s*<A>\s*(.*?)\s*(?=<Q>|$)"

# Find all matches of the pattern in the data
qa_pairs = re.findall(pattern, data)
print(qa_pairs)

[('Q1', 'Afefe1'), ('Q2 fef', 'A2')]


In [5]:
from datasets import Dataset, DatasetDict, load_dataset
dataset = load_dataset("tkdrnjs0621/musique_ans_processed2", split="test")

Generating test split:   0%|          | 0/2417 [00:00<?, ? examples/s]

In [2]:
from rapidfuzz import fuzz

def fuzzy_contains(main_string, sub_string, threshold=70):
    """
    Check if sub_string approximately exists in main_string based on a threshold.

    Parameters:
    - main_string (str): The main text to search within.
    - sub_string (str): The substring to search for within the main text.
    - threshold (int): The similarity threshold (0-100). Default is 70.

    Returns:
    - bool: True if a match above the threshold is found, False otherwise.
    """
    # Break the main_string into substrings of similar length to the sub_string
    for i in range(len(main_string) - len(sub_string) + 1):
        substring = main_string[i : i + len(sub_string)]
        # Compare substring similarity with the target sub_string
        if fuzz.ratio(substring, sub_string) >= threshold:
            return True
    return False

In [34]:
from transformers import AutoTokenizer, AutoModel
import torch

model_name = "facebook/contriever"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def encode_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        return model(**inputs).last_hidden_state.mean(dim=1)

def retrieve_knowledge_(knowledge_dictionary, query_question):
    texts = list(knowledge_dictionary.keys())

    query_embedding = encode_text(query_question)
    text_embeddings = torch.vstack([encode_text(text) for text in texts])

    similarities = torch.nn.functional.cosine_similarity(query_embedding, text_embeddings)
    top_text = texts[similarities.argmax().item()]
    top_text2 = list(knowledge_dictionary.values())[similarities.argmax().item()]
    return top_text2, top_text

/home/tkdrnjs0621/miniconda3/envs/torch_241/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [36]:
import re
def retrieve_knowledge(query, context):
    return retrieve_knowledge_({a:b for a,b in context}, query)
    # ls = []
    # for a in context:
    #     t = re.sub(r"\s*\(.*\)", "", a[0])
    #     if(fuzzy_contains(query,t,90)):
    #         ls.append((a[1],a[0]))
    # return [tmp[0] for tmp in ls], [tmp[1] for tmp in ls]
for i in range(10):
    data = dataset[i]

    print(data["qa_pairs"][0][0], data["answer"])
    print(retrieve_knowledge(data["qa_pairs"][0][0],data["merged_context"])[1])

Who is the Green performer? Miquette Giraudy
The Green Album
What company distributed the film UHF? Mike Medavoy
UHF (film)
What is Ciudad Deportiva? Tamaulipas
Ciudad Deportiva (Nuevo Laredo)
Who is Ulrich Walter's employer? Cologne
Ulrich Walter
What is the manufacturer of the Learjet 60? Bombardier Inc.
Learjet 60
Who is Caroline LeRoy's spouse? Fletcher Webster
Leroy
Who is Philippe, Duke of Orléans the child of? Marie de' Medici
Philippe II, Duke of Orléans
What is European Movement Germany a member of? European integration
European Movement Germany
What is the name of the owner of Empire Sports Network? Time Warner Cable
Empire Sports Network
Where is Lago District located? Cabo Delgado Province
Lago District


In [37]:
datasetx = load_dataset("tkdrnjs0621/musique_ans_processed2",  split="test")

In [39]:
data = datasetx[20]

print(data["question"],data["answer"])
print(data["qa_pairs"][0])
for title,v in data["merged_context"]:
    print(title)
    print(v)

What record label is the performer of Almost Made Ya signed to? Derrty Entertainment
['Who is the performer of "Almost Made Ya"?', '$1']
Something Personal
Something Personal is an album. Something Personal is by American jazz pianist Jack Wilson. Something Personal features performances recorded and released on the Blue Note label in 1967.
Jack Wilson
Jack Wilson is an American jazz pianist. Jack Wilson performed on the album Something Personal. Jack Wilson is featured on the album Something Personal.
Blue Note label
The Blue Note label released the album Something Personal in 1967. The Blue Note label recorded and released performances on the album Something Personal.
1967
1967 is the year when the album Something Personal was recorded and released on the Blue Note label.
American jazz pianist
Jack Wilson is an American jazz pianist who performed on the album Something Personal.
Groovin' with Golson
Groovin' with Golson is the sixth album by Benny Golson. Groovin' with Golson feature

In [11]:
from datasets import load_from_disk

a = load_from_disk("/home/tkdrnjs0621/work/multihopqa_v1/inference/subset_20s/context_conversion")
print(a[1])
print(a[1]["output"])

{'original_id': '2hop__569582_304416', 'title': "Groovin' with Golson", 'paragraph_text': "Groovin' with Golson is the sixth album by saxophonist Benny Golson featuring performances recorded in 1959 and originally released on the New Jazz label.", 'messages': [{'content': 'You are a language model that organizes the given text in an entity-based manner. Named entities should be extracted, and then the explanation about the entities should be listed. Note that the explanation should be very detailed, covering all information about the entity from the text, so information regarding two or more entities can be repeteadly mention in each entity explanation.', 'role': 'system'}, {'content': '[Topic]\nDarlene Remembers Duke, Jonathan Plays Fats\n[Text]\nDarlene Remembers Duke, Jonathan Plays Fats is a 1982 album by Jo Stafford and Paul Weston in which they perform in character as Jonathan and Darlene Edwards. The duo put their own unique interpretation on the music of Duke Ellington and Fats

In [14]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import nltk
from nltk.tokenize import sent_tokenize  # For sentence tokenization
from collections import defaultdict
# Load the tokenizer and model for NER

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)

# Input text
text = """Felix R. de Zoysa is the founding Chairman of Stafford Motor Company, Stafford International School and Atlas Hall in Sri Lanka. De Zoysa was the head of Auto & General Agencies who held the distributorship of Honda motor vehicles in Sri Lanka in 1960s. After the incorporation of both companies, Stafford Motor Company became the sole distributor for Honda motor vehicles. He was also nicknamed "Mr. Honda" of Sri Lanka."""

# Split text into sentences
sentences = sent_tokenize(text)

# Initialize dictionary for storing entity-specific text
entity_summaries = defaultdict(list)

# Process each sentence individually for entity recognition
for sentence in sentences:
    entities = nlp(sentence)
    print(entities)
    found_entities = [ent['word'] for ent in entities]
    
    # Add sentence to each identified entity in that sentence
    for entity in found_entities:
        entity_summaries[entity].append(sentence)

# Display results
for entity, summary in entity_summaries.items():
    print(f"<{entity}>: {' '.join(summary)}\n")

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/home/tkdrnjs0621/miniconda3/envs/torch_241/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'entity': 'B-PER', 'score': np.float32(0.99968755), 'index': 1, 'word': 'Felix', 'start': 0, 'end': 5}, {'entity': 'I-PER', 'score': np.float32(0.9983315), 'index': 2, 'word': 'R', 'start': 6, 'end': 7}, {'entity': 'I-PER', 'score': np.float32(0.9972046), 'index': 3, 'word': '.', 'start': 7, 'end': 8}, {'entity': 'I-PER', 'score': np.float32(0.99959344), 'index': 4, 'word': 'de', 'start': 9, 'end': 11}, {'entity': 'I-PER', 'score': np.float32(0.999482), 'index': 5, 'word': 'Z', 'start': 12, 'end': 13}, {'entity': 'I-PER', 'score': np.float32(0.9830192), 'index': 6, 'word': '##oy', 'start': 13, 'end': 15}, {'entity': 'I-PER', 'score': np.float32(0.9981741), 'index': 7, 'word': '##sa', 'start': 15, 'end': 17}, {'entity': 'B-ORG', 'score': np.float32(0.9994248), 'index': 13, 'word': 'Stafford', 'start': 46, 'end': 54}, {'entity': 'I-ORG', 'score': np.float32(0.99921644), 'index': 14, 'word': 'Motor', 'start': 55, 'end': 60}, {'entity': 'I-ORG', 'score': np.float32(0.9992679), 'index': 1